# Proyecto 1

**Andres Chaparro - 202111146** 

**Juan Esteban Lopez Ulloa - 202021417**

**Juan Diego Lozano Guerrero - 202122869**

# 1. Carga de datos (csv)
Se cargan la base de datos (CSV) para trabajar con estos

In [6]:

import importlib.util

def install_if_missing(package):
    if importlib.util.find_spec(package) is None:
        !pip install {package}

install_if_missing('contractions')
install_if_missing('inflect')
install_if_missing('scikit-plot')

In [7]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata
%pip install contractions
# librería para manejar las flexiones gramaticales en el idioma inglés.
%pip install inflect
# Manejo de gráficas
%pip install scikit-plot

import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/juanl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/juanl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/juanl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Cargar los datos de los archivos CSV
direccionTrain = 'fake_news_spanish.csv'
direccionValidation = 'fake_news_test.csv'
data_original_0 = pd.read_csv(direccionTrain, sep=";", encoding='utf-8')
data_validation = pd.read_csv(direccionValidation, sep=";", encoding='utf-8')
data_original_0.shape, data_validation.shape

((57063, 5), (1000, 4))

# 2. Entendimientos de los datos

In [10]:
data_original = data_original_0.copy()
data_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57063 entries, 0 to 57062
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           57063 non-null  object
 1   Label        57063 non-null  int64 
 2   Titulo       57047 non-null  object
 3   Descripcion  57063 non-null  object
 4   Fecha        57063 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


In [11]:
from scipy import stats as st

textos = data_original.copy()
textos['Conteo Titulo'] = [len(x) if pd.notnull(x) else 0 for x in textos['Titulo']]
textos['Max Titulo'] = [[max([len(x) for x in i.split(' ')])][0] if pd.notnull(i) else 0 for i in textos['Titulo']]
textos['Min Titulo'] = [[min([len(x) for x in i.split(' ')])][0] if pd.notnull(i) else 0  for i in textos['Titulo']]


In [12]:
titulo_stats = pd.DataFrame({
    'Estadística': ['Conteo Titulo', 'Max Titulo', 'Min Titulo'],
    'count': [textos['Conteo Titulo'].count(), textos['Max Titulo'].count(), textos['Min Titulo'].count()],
    'mean': [textos['Conteo Titulo'].mean(), textos['Max Titulo'].mean(), textos['Min Titulo'].mean()],
    'std': [textos['Conteo Titulo'].std(), textos['Max Titulo'].std(), textos['Min Titulo'].std()],
    'min': [textos['Conteo Titulo'].min(), textos['Max Titulo'].min(), textos['Min Titulo'].min()],
    '25%': [textos['Conteo Titulo'].quantile(0.25), textos['Max Titulo'].quantile(0.25), textos['Min Titulo'].quantile(0.25)],
    '50%': [textos['Conteo Titulo'].median(), textos['Max Titulo'].median(), textos['Min Titulo'].median()],
    '75%': [textos['Conteo Titulo'].quantile(0.75), textos['Max Titulo'].quantile(0.75), textos['Min Titulo'].quantile(0.75)],
    'max': [textos['Conteo Titulo'].max(), textos['Max Titulo'].max(), textos['Min Titulo'].max()]
})

titulo_stats


,Estadística,count,mean,std,min,25%,50%,75%,max
0,Conteo Titulo,57063,94.769413,21.881494,0,80.0,95.0,111.0,255
1,Max Titulo,57063,10.884338,2.028453,0,9.0,11.0,12.0,40
2,Min Titulo,57063,1.392443,0.534362,0,1.0,1.0,2.0,40


Como se puede observar para los titulos de las noticias en promedio estas tienen 94 letras siendo la noticia mas larga con 255 letras, en cuando a palabras en promedio cada titulo tiene como palabra maxima una de 10 letras y como palabra minima una de 1 letra

In [13]:
textos['Conteo Descripcion'] = [len(x) if pd.notnull(x) else 0 for x in textos['Descripcion']]
textos['Max Descripcion'] = [[max([len(x) for x in i.split(' ')])][0] if pd.notnull(i) else 0 for i in textos['Descripcion']]
textos['Min Descripcion'] = [[min([len(x) for x in i.split(' ')])][0] if pd.notnull(i) else 0  for i in textos['Descripcion']]

In [14]:
# Crear una tabla con las descripciones de las estadísticas
descripcion_stats = pd.DataFrame({
    'Estadística': ['Conteo Descripcion', 'Max Descripcion', 'Min Descripcion'],
    'count': [textos['Conteo Descripcion'].count(), textos['Max Descripcion'].count(), textos['Min Descripcion'].count()],
    'mean': [textos['Conteo Descripcion'].mean(), textos['Max Descripcion'].mean(), textos['Min Descripcion'].mean()],
    'std': [textos['Conteo Descripcion'].std(), textos['Max Descripcion'].std(), textos['Min Descripcion'].std()],
    'min': [textos['Conteo Descripcion'].min(), textos['Max Descripcion'].min(), textos['Min Descripcion'].min()],
    '25%': [textos['Conteo Descripcion'].quantile(0.25), textos['Max Descripcion'].quantile(0.25), textos['Min Descripcion'].quantile(0.25)],
    '50%': [textos['Conteo Descripcion'].median(), textos['Max Descripcion'].median(), textos['Min Descripcion'].median()],
    '75%': [textos['Conteo Descripcion'].quantile(0.75), textos['Max Descripcion'].quantile(0.75), textos['Min Descripcion'].quantile(0.75)],
    'max': [textos['Conteo Descripcion'].max(), textos['Max Descripcion'].max(), textos['Min Descripcion'].max()]
})

descripcion_stats

,Estadística,count,mean,std,min,25%,50%,75%,max
0,Conteo Descripcion,57063,243.448697,273.253727,33,171.0,221.0,267.0,14019
1,Max Descripcion,57063,12.711915,2.123055,6,11.0,13.0,14.0,69
2,Min Descripcion,57063,1.165554,0.384203,0,1.0,1.0,1.0,3


Como se puede observar para las descripciones de las noticias en promedio estas tienen 243 letras siendo la noticia mas larga con 14019 letras, en cuando a palabras en promedio cada titulo tiene como palabra maxima una de 12 letras y como palabra minima una de 1 letra

# 3. Limpieza y Preparación de los datos

## 3.1 Valores Nulos

In [15]:
data_original.isnull().sum()

ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64

In [16]:
data_original.dropna(inplace=True)
data_original.shape  

(57047, 5)

Se eliminaron los valores nulos de los datos para garantizar la integridad y calidad del análisis. La presencia de datos faltantes podría afectar la precisión de los modelos y generar inconsistencias en los resultados. Al remover estas entradas, se evitan sesgos y se asegura que las conclusiones obtenidas sean más confiables y representativas del conjunto de datos.

## 3.2 Valores duplicados

In [17]:
duplicados = data_original.duplicated().sum()
print(f"Cantidad de valores duplicados: {duplicados}")

Cantidad de valores duplicados: 445


In [18]:
data_original.drop_duplicates(inplace=True)
data_original.shape

(56602, 5)

Se eliminaron los valores duplicados para evitar redundancias y asegurar la calidad del análisis. La presencia de datos repetidos podría distorsionar los resultados y afectar la precisión de los modelos. Al remover estas entradas, se garantiza que cada observación sea única, lo que mejora la fiabilidad y representatividad del conjunto de datos.

## 3.3 Contracciones

Este código reemplaza las abreviaciones en las columnas `Titulo` y `Descripcion` del DataFrame `data_original` utilizando un diccionario con equivalencias en español. Esto ayuda a estandarizar el texto y mejorar su comprensión en análisis de datos. Por ejemplo, si en el texto aparece *"q"*, se reemplaza por *"que"*, o *"xq"* por *"porque"*, evitando ambigüedades y facilitando el procesamiento posterior.

In [19]:
import pandas as pd
import re
import unicodedata

abreviaciones = {
    "q": "que",
    "xq": "porque",
    "d": "de",
    "tb": "también",
    "mñn": "mañana",
    "uds": "ustedes",
    "pa": "para",
    "st": "esto",
    "ntp": "no te preocupes",
    "msj": "mensaje"
}

def limpiar_texto(texto):
    cambios = 0  # Contador de cambios
    if pd.notnull(texto):
        original = texto  # Guardar el texto original
        texto = texto.lower()
        texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
        
        palabras = texto.split()
        nuevas_palabras = []
        for p in palabras:
            if p in abreviaciones:
                cambios += 1  # Contar si hubo un cambio
            nuevas_palabras.append(abreviaciones.get(p, p))
        texto = " ".join(nuevas_palabras)

        texto = re.sub(r'[^a-zA-Z\s]', '', texto)
        
        if texto != original:
            cambios += 1  # Contar si hubo otro cambio en la limpieza final
            
        return texto, cambios
    return texto, 0


In [20]:
data_original_2 = data_original.copy()
data_original_2[['Titulo', 'Cambios_Titulo']] = data_original_2['Titulo'].apply(lambda x: pd.Series(limpiar_texto(x)))
data_original_2[['Descripcion', 'Cambios_Descripcion']] = data_original_2['Descripcion'].apply(lambda x: pd.Series(limpiar_texto(x)))

# Contar el total de cambios realizados
total_cambios = data_original_2['Cambios_Titulo'].sum() + data_original_2['Cambios_Descripcion'].sum()
print(f"Total de cambios realizados: {total_cambios}")

# Mostrar algunos ejemplos de cambios
data_original_2[['Titulo', 'Cambios_Titulo', 'Descripcion', 'Cambios_Descripcion']].head(5)

Total de cambios realizados: 113225


,Titulo,Cambios_Titulo,Descripcion,Cambios_Descripcion
0,the guardian va con sanchez europa necesita qu...,1,el diario britanico publico este pasado jueves...,1
1,revelan que el gobierno negocio la liberacion ...,1,revelan que el gobierno negocio la liberacion ...,1
2,el ahora o nunca de joan fuster sobre el estat...,1,el valencianismo convoca en castello su fiesta...,1
3,iglesias alienta a yolanda diaz erc y eh bildu...,1,en politica igual que hay que negociar con los...,1
4,puigdemont no seria ninguna tragedia una repet...,1,en una entrevista en el punt avui el lider de ...,1


In [21]:
data_original_3 = data_original_2.copy()
data_original_3['Titulo'] = data_original_3['Titulo'].apply(contractions.fix) 
data_original_3['Descripcion'] = data_original_3['Descripcion'].apply(contractions.fix) 

Aca se aplican igualmente contracciones pero esta vez para lenguaje de ingles cambiando palabras como don't en do not. Esto se hace con la finalidad de reducir la cantidad de tokens para despues no tener un entrenamiento en el modelo muy pesado

## 3.4 Tokenización

In [22]:
def tokenizar_texto(df):  
    df["Tokens_Titulo"] = df["Titulo"].astype(str).apply(lambda x: x.split())  
    df["Tokens_Descripcion"] = df["Descripcion"].astype(str).apply(lambda x: x.split())  
    return df  

In [23]:
data_original_4 = data_original_3.copy()  
data_original_4.drop(columns=["Cambios_Titulo", "Cambios_Descripcion"], inplace=True)
data_original_4 = tokenizar_texto(data_original_4)  
data_original_4.head(3)  

,ID,Label,Titulo,Descripcion,Fecha,Tokens_Titulo,Tokens_Descripcion
0,ID,1,the guardian va con sanchez europa necesita qu...,el diario britanico publico este pasado jueves...,02/06/2023,"[the, guardian, va, con, sanchez, europa, nece...","[el, diario, britanico, publico, este, pasado,..."
1,ID,0,revelan que el gobierno negocio la liberacion ...,revelan que el gobierno negocio la liberacion ...,01/10/2023,"[revelan, que, el, gobierno, negocio, la, libe...","[revelan, que, el, gobierno, negocio, la, libe..."
2,ID,1,el ahora o nunca de joan fuster sobre el estat...,el valencianismo convoca en castello su fiesta...,25/04/2022,"[el, ahora, o, nunca, de, joan, fuster, sobre,...","[el, valencianismo, convoca, en, castello, su,..."


Se paralelizaron los títulos y las descripciones mediante tokenización para descomponer el texto en unidades más pequeñas, lo que facilita su análisis y procesamiento. Esto permite una mejor manipulación de los datos, mejora el rendimiento en tareas de procesamiento del lenguaje natural y ayuda a extraer información relevante de manera más eficiente.

## 3.5 Limpieza de los tokens

In [24]:
import string

def to_lowercase(words):
    return [word.lower() for word in words]

def replace_numbers(words):
    return ["NUM" if word.isdigit() else word for word in words]

def remove_punctuation(words):
    return [word.translate(str.maketrans('', '', string.punctuation)) for word in words]

def remove_non_ascii(words):
    return [''.join(char for char in word if ord(char) < 128) for word in words]

def remove_stopwords(words):
    stopwords = {"lo", "sus","este", "esta","como", "no", "mas","del","para","se","al","su","ha","que","el", "la", "los", "las", "de", "y", "en", "a", "un", "una", "es", "por", "con", "the"}  # Agrega más si es necesario
    return [word for word in words if word not in stopwords]

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words


In [25]:
data_original_5 = data_original_4.copy()
data_original_5['Tokens_Titulo_Sin_Ruido']=data_original_5['Tokens_Titulo'].apply(preprocessing) 
data_original_5['Tokens_Descripcion_Sin_Ruido']=data_original_5['Tokens_Descripcion'].apply(preprocessing) 
data_original_5.head(1)

,ID,Label,Titulo,Descripcion,Fecha,Tokens_Titulo,Tokens_Descripcion,Tokens_Titulo_Sin_Ruido,Tokens_Descripcion_Sin_Ruido
0,ID,1,the guardian va con sanchez europa necesita qu...,el diario britanico publico este pasado jueves...,02/06/2023,"[the, guardian, va, con, sanchez, europa, nece...","[el, diario, britanico, publico, este, pasado,...","[guardian, va, sanchez, europa, necesita, apue...","[diario, britanico, publico, pasado, jueves, e..."


Se hace la definicion de una serie de funciones para la limpieza y normalización de texto. Primero, convierte todas las palabras a minúsculas, reemplaza los números por la etiqueta "NUM", elimina signos de puntuación y caracteres no ASCII, y finalmente, elimina palabras vacías comunes como artículos y preposiciones. La función `preprocessing` aplica estos pasos en secuencia para procesar una lista de palabras, asegurando que el texto esté limpio y uniforme antes de ser analizado.

In [26]:
# Instalar spaCy y descargar el modelo en español
# !pip install spacy
# !python -m spacy download es_core_news_sm


In [27]:
data_original_6 = data_original_5.copy()
data_original_6['Tokens_Combinados_Sin_Ruido'] = data_original_6['Tokens_Titulo_Sin_Ruido'] + data_original_6['Tokens_Descripcion_Sin_Ruido']
data_original_6['Tokens_Combinados_Sin_Ruido'] = data_original_6['Tokens_Combinados_Sin_Ruido'].apply(lambda x: sorted(set(x), key=x.index))
data_original_6['Tokens_Combinados_Sin_Ruido'].head(3)


0    [guardian, va, sanchez, europa, necesita, apue...
1    [revelan, gobierno, negocio, liberacion, mirel...
2    [ahora, o, nunca, joan, fuster, sobre, estatut...
Name: Tokens_Combinados_Sin_Ruido, dtype: object

Se conbina los tokens del título y la descripción para crear una representación más completa del contenido, eliminando el ruido. Al juntar los tokens y eliminar duplicados, se obtiene una lista única de palabras clave que representan mejor el contexto del texto, manteniendo el orden original. Esto mejora el análisis al concentrarse en las palabras relevantes.

In [28]:
"""import spacy

# Cargar el modelo de español en spaCy
nlp = spacy.load("es_core_news_sm")

def stem_words(words):
    'Aplica stemming eliminando sufijos comunes en español.'
    suffixes = ["ando", "iendo", "ar", "er", "ir", "ado", "ido", "ción", "ciones", "mente", "oso", "osa", "eza", "ico", "ica"]
    stemmed_words = []
    
    for word in words:
        for suffix in suffixes:
            if word.endswith(suffix):
                word = word[:-len(suffix)]
                break  # Si encuentra un sufijo, lo elimina y sigue con la siguiente palabra
        stemmed_words.append(word)
    
    return stemmed_words

def lemmatize_words(words):
    'Aplica lematización con spaCy.'
    doc = nlp(" ".join(words))  # Convertir la lista de palabras a texto y procesarla
    return [token.lemma_ for token in doc]  # Extraer las lemas

def stem_and_lemmatize(words):
    'Aplica primero lematización y luego stemming para reducir aún más la variabilidad.'
    lemmas = lemmatize_words(words)
    stems = stem_words(lemmas)
    stems = [stem for stem in stems if stem]
    return stems  # Elimina duplicados manteniendo el orden"""




'import spacy\n\n# Cargar el modelo de español en spaCy\nnlp = spacy.load("es_core_news_sm")\n\ndef stem_words(words):\n    \'Aplica stemming eliminando sufijos comunes en español.\'\n    suffixes = ["ando", "iendo", "ar", "er", "ir", "ado", "ido", "ción", "ciones", "mente", "oso", "osa", "eza", "ico", "ica"]\n    stemmed_words = []\n    \n    for word in words:\n        for suffix in suffixes:\n            if word.endswith(suffix):\n                word = word[:-len(suffix)]\n                break  # Si encuentra un sufijo, lo elimina y sigue con la siguiente palabra\n        stemmed_words.append(word)\n    \n    return stemmed_words\n\ndef lemmatize_words(words):\n    \'Aplica lematización con spaCy.\'\n    doc = nlp(" ".join(words))  # Convertir la lista de palabras a texto y procesarla\n    return [token.lemma_ for token in doc]  # Extraer las lemas\n\ndef stem_and_lemmatize(words):\n    \'Aplica primero lematización y luego stemming para reducir aún más la variabilidad.\'\n    lem

In [29]:
data_original_7 = data_original_6.copy()

#data_original_7['Tokens_Combinados_Sin_Ruido'] = data_original_7['Tokens_Combinados_Sin_Ruido'].apply(stem_and_lemmatize)


Se implementa un proceso de lematización y stemming para reducir la variabilidad de las palabras en un texto. Primero, se lematizan las palabras utilizando el modelo de spaCy, lo que permite reducir las palabras a su forma base. Luego, se aplica un proceso de stemming, eliminando sufijos comunes del idioma español, lo que ayuda a generalizar aún más las palabras. La combinación de ambos métodos mejora el análisis al obtener una representación más uniforme de las palabras, facilitando tareas como la clasificación o el análisis semántico.

In [30]:
x_data, y_data = data_original_6['Tokens_Combinados_Sin_Ruido'],data_original_6['Label']
x_data.shape, y_data.shape

((56602,), (56602,))

In [31]:
# Verifica el tipo de los elementos en x_data
print(type(x_data))
print(type(x_data[0]))

# Convierte listas de palabras en strings si es necesario
x_data_text = [" ".join(words) if isinstance(words, list) else str(words) for words in x_data]

# Aplica CountVectorizer
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(x_data_text)

print(X_dummy.shape)


<class 'pandas.core.series.Series'>
<class 'list'>
(56602, 48689)


In [32]:
from sklearn.model_selection import train_test_split

# Suponiendo que tienes etiquetas en 'y_data' (si no, usa solo X_dummy)
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y_data, test_size=0.2, random_state=42, shuffle=True)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((45281, 48689), (11321, 48689), (45281,), (11321,))

In [33]:
import pandas as pd
from collections import Counter

# Filtrar los datos para las clases 0 y 1
data_0 = data_original_6[data_original_6['Label'] == 0]
data_1 = data_original_6[data_original_6['Label'] == 1]

# Unir todas las listas de tokens en una sola para cada clase
tokens_0 = [token for sublist in data_0['Tokens_Combinados_Sin_Ruido'] for token in sublist]
tokens_1 = [token for sublist in data_1['Tokens_Combinados_Sin_Ruido'] for token in sublist]

# Contar las frecuencias de las palabras (tokens)
contador_0 = Counter(tokens_0)
contador_1 = Counter(tokens_1)

# Mostrar las 10 palabras más frecuentes para cada clase
top_10_0 = contador_0.most_common(10)
top_10_1 = contador_1.most_common(10)

# Convertir a DataFrame para mejor presentación
df_top_10_0 = pd.DataFrame(top_10_0, columns=['Palabra', 'Frecuencia'])
df_top_10_1 = pd.DataFrame(top_10_1, columns=['Palabra', 'Frecuencia'])

print("Top 10 palabras más frecuentes para la clase 0:")
print(df_top_10_0)

print("\nTop 10 palabras más frecuentes para la clase 1:")
print(df_top_10_1)


Top 10 palabras más frecuentes para la clase 0:
      Palabra  Frecuencia
0    gobierno        7031
1   catalunya        5124
2         per        4476
3  iniciativa        4466
4        equo        4217
5        vers        4197
6       sobre        3506
7  presidente        2973
8      contra        2962
9     partido        2889

Top 10 palabras más frecuentes para la clase 1:
      Palabra  Frecuencia
0    gobierno        8620
1          pp        6400
2       sobre        4577
3     sanchez        4299
4      contra        4130
5        psoe        4090
6     podemos        3998
7      madrid        3615
8  presidente        3371
9     partido        3330


Se puede observar que las palabras más frecuentes en ambas clases tienen similitudes, pero también diferencias clave. En ambas categorías, términos como *"no"*, *"gobierno"*, *"como"*, *"sobre"* y *"más"* aparecen con alta frecuencia, lo que indica que estos términos pueden ser comunes en el lenguaje político y en la estructura de las noticias en general. Sin embargo, en la clase 0, se destacan palabras como *"catalunya"*, *"iniciativa"*, y *"equo"*, que pueden estar relacionadas con noticias de carácter regional o político específico. Por otro lado, en la clase 1, términos como *"pp"*, *"sanchez"*, *"contra"* y *"psoe"* sugieren una mayor presencia de menciones a partidos políticos y confrontaciones políticas. Este análisis es útil para la organización, ya que permite identificar patrones lingüísticos distintivos en las noticias falsas y auténticas, lo que podría mejorar los modelos de detección y facilitar la comprensión del contenido manipulado.

# 4. Modelos predictivos

## 4.1 Funcion de Filtrado

Se crea una funcion que filtra los datos que entran sin modificar y crea el proceso de limpieza y el proceso de tokenizacion haciendo que queden los datos como entrada de tokenz para los modelos de machine learning.

In [34]:
def FiltrarTexto(df, vectorizer=dummy):
    texto = df.copy()
    texto.dropna(inplace=True)
    texto.drop_duplicates(inplace=True)
    texto[['Titulo', 'Cambios_Titulo']] = texto['Titulo'].apply(lambda x: pd.Series(limpiar_texto(x)))
    texto[['Descripcion', 'Cambios_Descripcion']] = texto['Descripcion'].apply(lambda x: pd.Series(limpiar_texto(x)))
    texto['Titulo'] = texto['Titulo'].apply(contractions.fix) 
    texto['Descripcion'] = texto['Descripcion'].apply(contractions.fix) 
    texto = tokenizar_texto(texto)
    texto['Tokens_Titulo_Sin_Ruido']=texto['Tokens_Titulo'].apply(preprocessing) 
    texto['Tokens_Descripcion_Sin_Ruido']=texto['Tokens_Descripcion'].apply(preprocessing) 
    texto['Tokens_Combinados_Sin_Ruido'] = texto['Tokens_Titulo_Sin_Ruido'] + texto['Tokens_Descripcion_Sin_Ruido']
    texto['Tokens_Combinados_Sin_Ruido'] = texto['Tokens_Combinados_Sin_Ruido'].apply(lambda x: sorted(set(x), key=x.index))
    x_datos = texto['Tokens_Combinados_Sin_Ruido']
    x_datos_text = [" ".join(words) if isinstance(words, list) else str(words) for words in x_datos]
    X_dummy_filtrado = vectorizer.transform(x_datos_text)
    return X_dummy_filtrado

## 4.2 Regresión Logistica (Andres)

### Construcción del modelo con búsqueda de hiperparámetros.

Se hace una iteración sobre los parametros del modelo para escoger los mejores parametros para el modelo

In [ ]:
from sklearn.model_selection import KFold 
# Fijemos el número de particiones. Utilizaremos K = 5.
particiones_regresion_logistica = KFold(n_splits=3, shuffle=True, random_state = 0)

Establecemos el espacio de búsqueda para los hiperparámetros que deseamos ajustar. 

In [ ]:
param_grid_regresion_logistica = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'saga'],
}


El parámetro C ajusta la regularización inversa, donde valores más pequeños aplican una mayor penalización para evitar el sobreajuste. La penalty define el tipo de regularización utilizada: l1 promueve la selección de variables, l2 reduce la magnitud de los coeficientes, y elasticnet combina ambas. El solver determina el algoritmo de optimización, con opciones como lbfgs para datasets grandes y liblinear para conjuntos pequeños. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

RegresionLogistica = LogisticRegression(max_iter=100)

# Ahora utilizamos GridSearch sobre el grid y particiones definidas
mejor_modelo = GridSearchCV(RegresionLogistica, param_grid_regresion_logistica, cv=particiones_regresion_logistica, n_jobs=-1)

# Ajuste del modelo
mejor_modelo.fit(X_train, y_train)


c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
12 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\andre\AppDat

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
# Podemos ver cuál fue el resultado de la búsqueda (mejores valores de hiperparámetros)
mejor_modelo.best_params_


{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

Estos son los parametros optimos para el modelo de regresion logistica

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Hacer predicciones
Regresion_Logistica_final = mejor_modelo.best_estimator_
y_pred = Regresion_Logistica_final.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Mostrar los resultados
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"ROC AUC: {roc_auc}")


Accuracy: 0.9156434943909548
Precision: 0.915576694411415
Recall: 0.940888956774095
F1-Score: 0.928060263653484
ROC AUC: 0.9109555801884719


Como se puede observar, el modelo de regresión logística presenta un **accuracy** de 0.90, lo que indica que el 90% de las predicciones realizadas son correctas, lo cual es un buen indicador general de su desempeño. Además, con una **precisión** de 0.90, el modelo muestra una alta capacidad para identificar correctamente las noticias verídicas sin generar demasiados falsos positivos. El **recall** de 0.93 es especialmente destacable, ya que muestra que el modelo es efectivo para detectar la mayoría de las noticias verídicas, lo cual es crucial para evitar que pasen desapercibidas. El **F1-Score**, que balancea precisión y recall, es de 0.92, lo que sugiere que el modelo tiene un buen equilibrio entre ambos. Finalmente, el **ROC AUC** de 0.90 muestra una buena capacidad del modelo para discriminar entre las noticias falsas y verídicas, indicando que el modelo tiene una sólida capacidad de predicción general. Estos resultados reflejan que el modelo es robusto y adecuado para la tarea de clasificación, con un desempeño muy eficiente en la identificación de noticias verídicas.

### Modelo de Regresión Logistica completo

Teniendo en cuenta los buenos resultados del modelo entrenado con el 80% de los datos se opto por hacer el modelo final con los datos completos

In [ ]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer

filtrar_transformer = FunctionTransformer(FiltrarTexto)

RegresionLogisticaCompleta = Pipeline([
    ("filtrar", filtrar_transformer),
    ('model', mejor_modelo.best_estimator_)  
])

# Entrenar el pipeline
RegresionLogisticaCompleta.fit(data_original_0, data_original_6['Label'])


Pipeline(steps=[('filtrar',
                 FunctionTransformer(func=<function FiltrarTexto at 0x00000276CC757060>)),
                ('model',
                 LogisticRegression(C=1, penalty='l1', solver='liblinear'))])

Dado que se obtuvo un muy buen modelo entrenado con en 80% de lso datos se decidio volver a crear un modelo pero esta vez entrenado con el 100% de los datos

In [ ]:
from joblib import dump
# Exportar el pipeline entrenado en formato .joblib
dump(RegresionLogisticaCompleta, "RegresionLogistica.joblib")
print("El pipeline ha sido exportado como 'RegresionLogistica.joblib' en el directorio actual")

El pipeline ha sido exportado como 'RegresionLogistica.joblib' en el directorio actual


In [ ]:
predicciones = RegresionLogisticaCompleta.predict(data_validation)

df_predicciones = pd.DataFrame(predicciones, columns=["Predicciones"])
df_resultado = pd.concat([data_validation, df_predicciones], axis=1)
df_resultado.to_excel("prediccionesRegresionLogistica.xlsx", index=False)
df_resultado.head()


,ID,Titulo,Descripcion,Fecha,Predicciones
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018,0.0
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023,1.0
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019,0.0
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020,1.0
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021,1.0


## 4.3 Random Forest (Andres)

### Construcción del modelo con búsqueda de hiperparámetros.

In [ ]:
from sklearn.model_selection import KFold 
# Fijemos el número de particiones. Utilizaremos K = 5.
particiones_Random_Forest = KFold(n_splits=3, shuffle=True, random_state = 0)

Establecemos el espacio de búsqueda para los hiperparámetros que deseamos ajustar. 

In [ ]:
param_grid_random_forest = {
    'n_estimators': [50, 100, 200],  # Número de árboles
    'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
}



El hiperparámetro `n_estimators` define la cantidad de árboles en el bosque, donde más árboles suelen mejorar el rendimiento pero aumentan el tiempo de entrenamiento. `max_depth` controla la profundidad máxima de cada árbol, con valores más altos permitiendo mayor complejidad pero riesgo de sobreajuste. `min_samples_split` establece el número mínimo de muestras necesarias para dividir un nodo, influyendo en la generalización del modelo, ya que valores más altos reducen la fragmentación excesiva.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

RandomForest = RandomForestClassifier(random_state=42)

# Ahora utilizamos GridSearch sobre el grid y particiones definidas
mejor_modelo_random_forest = GridSearchCV(RandomForest, param_grid_random_forest, cv=particiones_Random_Forest, n_jobs=-1)

# Ajuste del modelo
mejor_modelo_random_forest.fit(X_train, y_train)
# Podemos ver cuál fue el resultado de la búsqueda (mejores valores de hiperparámetros)
mejor_modelo_random_forest.best_params_



{'max_depth': None, 'n_estimators': 200}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RandomForest = Pipeline([
    ('model', mejor_modelo_random_forest.best_estimator_)  
    
])
RandomForest.fit(X_train, y_train)


Pipeline(steps=[('model',
                 RandomForestClassifier(n_estimators=200, random_state=42))])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = RandomForest.predict(X_test)
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(conf_matrix)


              precision    recall  f1-score   support

           0       0.93      0.85      0.89      4774
           1       0.90      0.95      0.92      6547

    accuracy                           0.91     11321
   macro avg       0.91      0.90      0.91     11321
weighted avg       0.91      0.91      0.91     11321

Matriz de confusión:
[[4052  722]
 [ 307 6240]]


Se puede observar que el modelo presenta un buen rendimiento global con una precisión de 0.91, lo que indica que el 91% de las predicciones son correctas. Para las noticias legítimas (clase 0), el modelo tiene una precisión de 0.92 y un recall de 0.84, lo que sugiere que, aunque identifica correctamente la mayoría de las noticias legítimas, hay un 16% de noticias legítimas que se clasifican erróneamente como falsas. En cuanto a las noticias falsas (clase 1), el modelo muestra un excelente desempeño con una precisión de 0.89 y un recall de 0.95, lo que significa que identifica la gran mayoría de las noticias falsas sin etiquetar demasiadas noticias legítimas como falsas. La matriz de confusión revela que, aunque hay algunos falsos positivos y negativos, el modelo está funcionando de manera efectiva para detectar noticias falsas, lo cual es crucial para la organización.

### Modelo de Random Forest Completo

Teniendo en cuenta los buenos resultados del modelo entrenado con el 80% de los datos se opto por hacer el modelo final con los datos completos

In [ ]:
from sklearn.preprocessing import FunctionTransformer

filtrar_transformer = FunctionTransformer(FiltrarTexto)

RandomForestCompleta = Pipeline([
    ("filtrar", filtrar_transformer),
    ('model', mejor_modelo_random_forest.best_estimator_)  

])

# Entrenar el pipeline
RandomForestCompleta.fit(data_original_0, data_original_6['Label'])


Pipeline(steps=[('filtrar',
                 FunctionTransformer(func=<function FiltrarTexto at 0x00000276CC757060>)),
                ('model',
                 RandomForestClassifier(n_estimators=200, random_state=42))])

Dado que se obtuvo un muy buen modelo entrenado con en 80% de lso datos se decidio volver a crear un modelo pero esta vez entrenado con el 100% de los datos

In [ ]:
from joblib import dump

# Exportar el pipeline entrenado en formato .joblib
dump(RandomForestCompleta, "RandomForest.joblib")
print("El pipeline ha sido exportado como 'RandomForest.joblib' en el directorio actual")

El pipeline ha sido exportado como 'RandomForest.joblib' en el directorio actual


In [ ]:
predicciones = RandomForestCompleta.predict(data_validation)

df_predicciones = pd.DataFrame(predicciones, columns=["Predicciones"])
df_resultado = pd.concat([data_validation, df_predicciones], axis=1)
df_resultado.to_excel("prediccionesRandomForest.xlsx", index=False)
df_resultado.head()


,ID,Titulo,Descripcion,Fecha,Predicciones
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018,0.0
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023,1.0
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019,0.0
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020,1.0
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021,1.0


### 4.4 Modelo multinomial Naive Bayes

In [ ]:
# Importar libreria para bernoilli de Naive Bayes
from sklearn.naive_bayes import BernoulliNB

# Crear el pipeline
bernoulliNB = Pipeline([
    
    ('model', BernoulliNB())  
])

# Entrenar el pipeline
bernoulliNB.fit(X_train, y_train)

Pipeline(steps=[('model', BernoulliNB())])

In [ ]:
from sklearn.naive_bayes import MultinomialNB

multinomialNB = Pipeline([
    
    ('model', MultinomialNB())  
])

multinomialNB.fit(X_train, y_train)

Pipeline(steps=[('model', MultinomialNB())])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Hacer predicciones
y_pred_mnb = multinomialNB.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_mnb)
precision = precision_score(y_test, y_pred_mnb)
recall = recall_score(y_test, y_pred_mnb)
f1 = f1_score(y_test, y_pred_mnb)
roc_auc = roc_auc_score(y_test, y_pred_mnb)

y_pred_bnb = bernoulliNB.predict(X_test)

accuracy_2 = accuracy_score(y_test, y_pred_bnb)
precision_2 = precision_score(y_test, y_pred_bnb)
recall_2 = recall_score(y_test, y_pred_bnb)
f1_2 = f1_score(y_test, y_pred_bnb)
roc_auc_2 = roc_auc_score(y_test, y_pred_bnb)

# Mostrar los resultados para los dos modelos
print("Multinomial Naive Bayes:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"ROC AUC: {roc_auc}")

print("\nBernoulli Naive Bayes:")
print(f"Accuracy: {accuracy_2}")
print(f"Precision: {precision_2}")
print(f"Recall: {recall_2}")
print(f"F1-Score: {f1_2}")
print(f"ROC AUC: {roc_auc_2}")


Multinomial Naive Bayes:
Accuracy: 0.8449783588022259
Precision: 0.8474477958236659
Recall: 0.892622575225294
F1-Score: 0.8694487837536264
ROC AUC: 0.83613114517444

Bernoulli Naive Bayes:
Accuracy: 0.845861672997085
Precision: 0.8379786036036037
Recall: 0.9092714220253552
F1-Score: 0.8721705369569995
ROC AUC: 0.8340869049800005


Se tiene un mejor F1-score en el modelo de Bernoulli por lo que se hara el entrenamiento con los datos completos con este modelo y este es el modelo que se exportará

In [ ]:
from sklearn.preprocessing import FunctionTransformer

filtrar_transformer = FunctionTransformer(FiltrarTexto)

bernoulliNBCompleta = Pipeline([
    ("filtrar", filtrar_transformer),
    ('model', BernoulliNB())  
])

# Entrenar el pipeline
bernoulliNBCompleta.fit(data_original_0, data_original_6['Label'])

Pipeline(steps=[('filtrar',
                 FunctionTransformer(func=<function FiltrarTexto at 0x000002A4300D7D80>)),
                ('model', BernoulliNB())])

Ahora se creara el archivo .joblib con el pipline creado

In [ ]:
from joblib import dump

# Exportar el pipeline entrenado en formato .joblib
dump(bernoulliNBCompleta, "BernoulliNavieBayes.joblib")
print("El pipeline ha sido exportado como 'BernoulliNavieBayes.joblib' en el directorio actual")

El pipeline ha sido exportado como 'BernoulliNavieBayes.joblib' en el directorio actual


In [ ]:
predicciones_bnb = bernoulliNBCompleta.predict(data_validation)

df_predicciones_bnb = pd.DataFrame(predicciones_bnb, columns=["Predicciones"])
df_resultado_bnb = pd.concat([data_validation, df_predicciones_bnb], axis=1)
df_resultado_bnb.to_excel("prediccionesBernoulliNaiveBayes.xlsx", index=False)
df_resultado_bnb.head()

,ID,Titulo,Descripcion,Fecha,Predicciones
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018,0.0
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023,1.0
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019,0.0
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020,0.0
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021,1.0


## 4.5 Support Vector Machine (Juan Diego)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm = Pipeline([
    ('scaler', StandardScaler(with_mean=False)), 
    ('svc', LinearSVC(dual=False, max_iter=10000))
])

#clf.fit(X_train, y_train)
svm.fit(X_train, y_train)



/Users/juanl/opt/anaconda3/envs/py39bi/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler(with_mean=False)),
                ('svc', LinearSVC(dual=False, max_iter=10000))])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Hacer predicciones
y_pred_svm = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_svm)
precision = precision_score(y_test, y_pred_svm)
recall = recall_score(y_test, y_pred_svm)
f1 = f1_score(y_test, y_pred_svm)
roc_auc = roc_auc_score(y_test, y_pred_svm)

# Mostrar los resultados
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"ROC AUC: {roc_auc}")

Accuracy: 0.7919795071106792
Precision: 0.8357898109580263
Recall: 0.7968535206965022
F1-Score: 0.8158573774337321
ROC AUC: 0.7910744352539906


### SVM completo

In [ ]:
from sklearn.preprocessing import FunctionTransformer

filtrar_transformer = FunctionTransformer(FiltrarTexto)

SVM_Completo = Pipeline([
    ("filtrar", filtrar_transformer),
    ('scaler', StandardScaler(with_mean=False)), 
    ('svc', LinearSVC(dual=False, max_iter=10000))
])

# Entrenar el pipeline
SVM_Completo.fit(data_original_0, data_original_6['Label'])


In [ ]:
from joblib import dump

# Exportar el pipeline entrenado en formato .joblib
dump(SVM_Completo, "SVM.joblib")
print("El pipeline ha sido exportado como 'SVM.joblib' en el directorio actual")

In [ ]:
predicciones = SVM_Completo.predict(data_validation)

df_predicciones = pd.DataFrame(predicciones, columns=["Predicciones"])
df_resultado = pd.concat([data_validation, df_predicciones], axis=1)
df_resultado.to_excel("prediccionesSVM.xlsx", index=False)
df_resultado.head()


## 4.6 Árboles de decisión (Juan Diego)

In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer

clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, X_train, y_train, cv=10)
clf.fit(X_train, y_train)
filtrar_transformer = FunctionTransformer(FiltrarTexto)

#Pipeline 
arbol = Pipeline([
    ("filtrar", filtrar_transformer),
    ('model', DecisionTreeClassifier(random_state=0))  
])
arbol.fit(data_original_0, data_original_6['Label'])


Pipeline(steps=[('filtrar',
                 FunctionTransformer(func=<function FiltrarTexto at 0x1461c4670>)),
                ('model', DecisionTreeClassifier(random_state=0))])

In [36]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.92      0.91      0.91      4774
           1       0.93      0.94      0.94      6547

    accuracy                           0.93     11321
   macro avg       0.93      0.92      0.93     11321
weighted avg       0.93      0.93      0.93     11321



In [37]:
predicciones = arbol.predict(data_validation)

df_predicciones = pd.DataFrame(predicciones, columns=["Predicciones"])
df_resultado = pd.concat([data_validation, df_predicciones], axis=1)
df_resultado.to_excel("prediccionesArbol.xlsx", index=False)
df_resultado.head()

,ID,Titulo,Descripcion,Fecha,Predicciones
0,ID,La mesa del congreso censura un encuentro inte...,"Portavoces de Ciudadanos, PNV, UPN, PSOE, Unid...",30/10/2018,0.0
1,ID,La brecha digital que dificulta el acceso de a...,No es la primera vez que los ciudadanos vulner...,15/03/2023,0.0
2,ID,PP apremia al EQUO a presentar una propuesta d...,El partido morado reprocha que los socialistas...,01/07/2019,0.0
3,ID,De soberano garante de la democracia a rey cor...,"La renuncia de Felipe VI a su herencia, proced...",16/03/2020,1.0
4,ID,El Gobierno aprobará este martes detraer los b...,El Ejecutivo también prorrogará la suspensión ...,13/09/2021,1.0
